In [1]:
import requests        
import json            
import pandas as pd    
from datetime import datetime as dt
pd.options.mode.chained_assignment = None

In [2]:
# Parameters required for API request
root_url = 'https://api.binance.com/api/v1/klines'
interval = '1h'
limit = '1000'

#Creating list of dates as API only allows for 1000 lines per request so we will get requests 1 month at a time
jul19 = round(dt.timestamp(dt(2019,7,1,0)))*1000 # Api requests time to be in timestamp format in milliseconds
aug19 = round(dt.timestamp(dt(2019,8,1,0)))*1000
sep19 = round(dt.timestamp(dt(2019,9,1,0)))*1000
oct19 = round(dt.timestamp(dt(2019,10,1,0)))*1000
nov19 = round(dt.timestamp(dt(2019,11,1,0)))*1000
dec19 = round(dt.timestamp(dt(2019,12,1,0)))*1000
jan20 = round(dt.timestamp(dt(2020,1,1,0)))*1000
feb20 = round(dt.timestamp(dt(2020,2,1,0)))*1000
mar20 = round(dt.timestamp(dt(2020,3,1,0)))*1000
apr20 = round(dt.timestamp(dt(2020,4,1,0)))*1000
may20 = round(dt.timestamp(dt(2020,5,1,0)))*1000
jun20 = round(dt.timestamp(dt(2020,6,1,0)))*1000
jul20 = round(dt.timestamp(dt(2020,7,1,0)))*1000
aug20 = round(dt.timestamp(dt(2020,8,1,0)))*1000
sep20 = round(dt.timestamp(dt(2020,9,1,0)))*1000
oct20 = round(dt.timestamp(dt(2020,10,1,0)))*1000
nov20 = round(dt.timestamp(dt(2020,11,1,0)))*1000
dec20 = round(dt.timestamp(dt(2020,12,1,0)))*1000
jan21 = round(dt.timestamp(dt(2021,1,1,0)))*1000
feb21 = round(dt.timestamp(dt(2021,2,1,0)))*1000
mar21 = round(dt.timestamp(dt(2021,3,1,0)))*1000
apr21 = round(dt.timestamp(dt(2021,4,1,0)))*1000
may21 = round(dt.timestamp(dt(2021,5,1,0)))*1000
jun21 = round(dt.timestamp(dt(2021,6,1,0)))*1000
jul21 = round(dt.timestamp(dt(2021,7,1,0)))*1000


dates = [jul19, aug19, sep19, oct19, nov19, dec19, jan20, feb20, mar20, apr20, may20, jun20,
         jul20, aug20, sep20, oct20, nov20, dec20, jan21, feb21, mar21, apr21, may21, jun21, jul21]

In [3]:
def api(pair, interval, startTime, endTime, limit):
    ''' Function calls Binance API to obtain historical data on cryptocurrency prices 
    Parameters: 
        pair(str) = trading pair str, valid options include DOGEUSDT, ETHUSDT, BTCUSDT etc...
        interval(str) = period interval between prices
        startTime(int) = start time period in millisecond timestamp format
        endTime(int) = end time period in millisecond timestamp fromat
        limit(str) = amount of lines requested, Binance API has a max request order of 1000
    Outputs:
        Dataframe with the histroical prices of requested cryptocurrency with data including opening price, closing price, price interval high and low, and trading volume
        '''
    #building API URL according to 
    url = root_url + '?symbol=' + symbol + '&interval=' + interval +'&startTime=' + str(startTime) +'&endTime=' + str(endTime) + '&limit=' + limit
    data = json.loads(requests.get(url).text)
    df = pd.DataFrame(data)
    df.columns = ['open_time',
                 'open', 'high', 'low', 'close', 'volume',
                 'close_time', 'qav', 'num_trades',
                 'taker_base_vol', 'taker_quote_vol', 'ignore']
    
    for i in range(len(df)):
        # Add one to round end time to the nearest hour
        df.close_time[i] = df.close_time[i]+1
        df.open[i] = float(df.open[i])
        df.high[i] = float(df.high[i])
        df.low[i] = float(df.low[i])
        df.close[i] = float(df.close[i])
        df.volume[i] = float(df.volume[i])
        df.num_trades[i] = float(df.num_trades[i])
        
        df.index = [dt.fromtimestamp(x/1000) for x in df.close_time]
    
    return df[['open', 'high', 'low', 'close', 'volume','num_trades']]

In [4]:
def get_price(dates,pair):
    df = pd.DataFrame()
    for i in range(len(dates)-1):
        df.drop(df.tail(1).index,inplace=True)
        df = df.append(api(pair,interval, dates[i], dates[i+1], limit))
    return df

In [97]:
#DOGE = get_price(dates,'DOGEUSDT')
#BTC = get_price(dates,'BTCUSDT')

In [111]:
DOGE

,open,high,low,close,volume,num_trades
2019-07-05 07:00:00,0.00449,0.0046,0.00376,0.003925,949430724.0,6487
2019-07-05 08:00:00,0.003915,0.003928,0.003765,0.003835,159004267.0,1529
2019-07-05 09:00:00,0.003845,0.00392,0.003814,0.003841,105517060.0,1241
2019-07-05 10:00:00,0.003837,0.003869,0.00371,0.003738,100804728.0,1213
2019-07-05 11:00:00,0.003728,0.003734,0.003636,0.003666,120261079.0,1416
...,...,...,...,...,...,...
2021-06-30 21:00:00,0.25182,0.25208,0.24485,0.24552,66497079.3,22834
2021-06-30 22:00:00,0.24555,0.24705,0.24286,0.24306,48553525.7,17177
2021-06-30 23:00:00,0.24305,0.24715,0.2423,0.24697,44887310.7,16267
2021-07-01 00:00:00,0.24697,0.248,0.24477,0.24702,35714148.5,15537


In [110]:
BTC

,open,high,low,close,volume,num_trades
2019-07-01 01:00:00,11078.79,11138.83,10935.41,10962.89,2974.791124,23942
2019-07-01 02:00:00,10960.4,11078.63,10900.0,10983.7,2740.122383,24911
2019-07-01 03:00:00,10983.7,11180.85,10950.0,11178.08,2712.777791,25602
2019-07-01 04:00:00,11178.13,11189.99,11072.62,11106.79,2185.825146,21262
2019-07-01 05:00:00,11107.17,11146.96,10902.59,10994.45,2559.981085,23928
...,...,...,...,...,...,...
2021-06-30 21:00:00,34936.43,34966.53,34223.15,34299.59,3787.824933,59229
2021-06-30 22:00:00,34299.58,34403.55,34120.0,34120.01,1797.404599,34643
2021-06-30 23:00:00,34120.01,34338.03,34061.16,34330.34,1935.266555,35895
2021-07-01 00:00:00,34330.34,34408.0,34170.46,34325.72,1379.833783,28123


In [99]:
BTC.to_csv('API Data/BTC_price.csv')

In [100]:
DOGE.to_csv('API Data/DOGE_price.csv')